In [ ]:
import os
import io
import wget
import pandas as pd
from PIL import Image

In [ ]:
%cd 
!python ./text_detection/test.py \
--trained_model ./saved_models/text_detection.pth \
--test_folder ./text_detection/test/test2

df = pd.read_csv('./text_detection/result/res_test2.txt',sep=',',names=['x1','y1','x2','y2','x3','y3','x4','y4'])
df['text_size'] = df['y3'] - df['y1']

line_list = []
text_size = df['text_size'].mean()
start = len(line_list)
for _ in range(len(df)):
    x1,y1,x2,y2,x3,y3,x4,y4,text_size = df.loc[_]
    box = x1,y1,x2,y2,x3,y3,x4,y4
    
    idx = 0
    
    if _ ==0:
        li = []
        li.append(box)
        line_list.append(li)
            
    for line in line_list[start:]:
        do_append = 'F'
        if _ ==0:
            break
        for i in line:
            if (abs(((y1 + y3) /2) - ((i[1] + i[5])/2)) < text_size):
                line.append(box)
                line.sort()
                do_append = 'T'
                break    
        if do_append == 'T':
            break
        else:
            idx += 1

    if idx == len(line_list[start:]):
        new_line = []
        new_line.append(box)
        line_list.append(new_line)
        
before_x_point = 0
before_y_point = 0
text_structure = []
idx = 0
for line in line_list:
    if idx ==0:
        new_paragraph = []
        new_paragraph.append(line)
    
    else:
        if (line[0][1] - before_y_point) > 1.5 *text_size:
            text_structure.append(new_paragraph)
            new_paragraph = []
            new_paragraph.append(line)
        else:
            new_paragraph.append(line)
         
    before_x_point = line[0][0]
    before_y_point = line[0][1]
         
    before_x_point = line[0][0]
    before_y_point = line[0][1]
    idx += 1
    
    if idx == len(line_list):
        text_structure.append(new_paragraph)
img = Image.open('./text_detection/test/test2/test2.jpg','r')

a = 0
k = 0
for paragraph in text_structure:
    path = './text_detection/result/test2_structure/'
    paragraph_path = path + 'paragraph' + str(k) + '/'
    os.mkdir(paragraph_path)
    for line in paragraph:
        line.sort()
        line_path = paragraph_path + 'line' + str(a) + '/'
        os.mkdir(line_path)
        b = 0
        for box in line:
            x1,y1,x2,y2,x3,y3,x4,y4 = box
            image = img.crop((x1,y1,x3,y3))
            image.save(line_path + str(b) +'.jpg')
            b += 1
        a += 1
    k += 1
    
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/ocr/My Project-a3f9a30ceabf.json"
client = vision.ImageAnnotatorClient()
path = './text_detection/test/test2/test2.jpg'
with io.open(path, 'rb') as image_file:
    content = image_file.read()
    
image = vision.types.Image(content=content)
response = client.text_detection(image=image)
texts = response.text_annotations
box = 0,0,0,0,0,0,0,0
pre_box = 0,0,0,0,0,0,0,0
result_text = []
a = ' '
for text in texts[1:]:
    for vertex in text.bounding_poly.vertices:
        vertices = (['{},{}'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
        x1,x2,x3,x4 = vertices[0].split(',')[0],vertices[1].split(',')[0],vertices[2].split(',')[0],vertices[3].split(',')[0]
        y1,y2,y3,y4 = vertices[0].split(',')[1],vertices[1].split(',')[1],vertices[2].split(',')[1],vertices[3].split(',')[1]
        break
    
    box = int(x1),int(y1),int(x2),int(y2),int(x3),int(y3),int(x4),int(y4)
    
    if (box[1] - pre_box[5]) >= 30:
        result_text.append(a)
        print('문단나누기')
        a = text.description
    else:
        a += text.description + ' '
    pre_box = box
result_text.append(a)